The 2 csv files we need are not getting read into BigQuery- even when column attributes are manually specified rather than automatically detected-, at least for now.

Excel is at least able to read the data in,
but cannot read in some of the date values correctly (some date values appear fine, while others get read in as "#########")

The goal is now to read in the csv files using Python, clean the data if necessary, extract the JSON values in each dataset to appropriate columns, and then re-output to csvs, and re-upload to BigQuery + Excel to see if that solves the issue.

Step 1: Read in CSVs

In [ ]:
import pandas as pd
import json

# Read the CSV files
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

# Clean the data (if necessary)
# ...

# Handle JSON data
# This is just an example. You'll need to replace 'json_column' with the actual column name.
# credits['json_column'] = credits['json_column'].apply(json.loads)

# Export to CSV
# merged.to_csv('merged.csv', index=False)

In [ ]:
# Print the first 20 records from the 'movies' DataFrame
print(movies.head(20))

       budget  \
0   237000000   
1   300000000   
2   245000000   
3   250000000   
4   260000000   
5   258000000   
6   260000000   
7   280000000   
8   250000000   
9   250000000   
10  270000000   
11  200000000   
12  200000000   
13  255000000   
14  225000000   
15  225000000   
16  220000000   
17  380000000   
18  225000000   
19  250000000   

                                                                                                                                    genres  \
0   [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]   
1                                           [{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]   
2                                             [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]   
3                 [{"id": 28, "name": "Action"}, {"id": 80, "name": "Crime

In [ ]:
# Print the first 20 records from the 'credits' DataFrame
print(credits.head(20))

    movie_id                                        title  \
0      19995                                       Avatar   
1        285     Pirates of the Caribbean: At World's End   
2     206647                                      Spectre   
3      49026                        The Dark Knight Rises   
4      49529                                  John Carter   
5        559                                 Spider-Man 3   
6      38757                                      Tangled   
7      99861                      Avengers: Age of Ultron   
8        767       Harry Potter and the Half-Blood Prince   
9     209112           Batman v Superman: Dawn of Justice   
10      1452                             Superman Returns   
11     10764                            Quantum of Solace   
12        58   Pirates of the Caribbean: Dead Man's Chest   
13     57201                              The Lone Ranger   
14     49521                                 Man of Steel   
15      2454     The Chr

In [ ]:
#function to parse JSON lists into actual lists:
def extract_json_values(df, columns, separator=', ', replace='no', key='name', new_column_name=None):
    # Function to convert JSON string to comma-separated string
    def convert_json(json_str):
        try:
            json_list = json.loads(json_str)
            values = [item[key] for item in json_list]
            return separator.join(values)
        except json.JSONDecodeError:
            return ''

    for column in columns:
        # Apply the function to the column
        new_col_name = new_column_name if new_column_name else column
        df[new_col_name + '_json' if replace == 'no' else new_col_name] = df[column].apply(convert_json)

    # Remove leading and trailing whitespace from all string columns
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    return df


In [ ]:
#parse the JSON list columns in the movies table:
movies_lists_extracted = extract_json_values(movies, ['genres', 'production_companies', 'keywords', 'production_countries', 'spoken_languages'], replace='yes')


In [ ]:
print(movies_lists_extracted.head(20))
#print(movies.head(20))

       budget                                       genres  \
0   237000000  Action, Adventure, Fantasy, Science Fiction   
1   300000000                   Adventure, Fantasy, Action   
2   245000000                     Action, Adventure, Crime   
3   250000000               Action, Crime, Drama, Thriller   
4   260000000           Action, Adventure, Science Fiction   
5   258000000                   Fantasy, Action, Adventure   
6   260000000                            Animation, Family   
7   280000000           Action, Adventure, Science Fiction   
8   250000000                   Adventure, Fantasy, Family   
9   250000000                   Action, Adventure, Fantasy   
10  270000000  Adventure, Fantasy, Action, Science Fiction   
11  200000000           Adventure, Action, Thriller, Crime   
12  200000000                   Adventure, Fantasy, Action   
13  255000000                   Action, Adventure, Western   
14  225000000  Action, Adventure, Fantasy, Science Fiction   
15  2250

In [ ]:
#extract names of cast from the credits dataset:
credits_lists_extracted = extract_json_values(credits, ['cast'], replace='yes')


In [ ]:
print(credits_lists_extracted.head(20))
#print(credits.head(20))

    movie_id                                        title  \
0      19995                                       Avatar   
1        285     Pirates of the Caribbean: At World's End   
2     206647                                      Spectre   
3      49026                        The Dark Knight Rises   
4      49529                                  John Carter   
5        559                                 Spider-Man 3   
6      38757                                      Tangled   
7      99861                      Avengers: Age of Ultron   
8        767       Harry Potter and the Half-Blood Prince   
9     209112           Batman v Superman: Dawn of Justice   
10      1452                             Superman Returns   
11     10764                            Quantum of Solace   
12        58   Pirates of the Caribbean: Dead Man's Chest   
13     57201                              The Lone Ranger   
14     49521                                 Man of Steel   
15      2454     The Chr

In [ ]:
# function to extract new columns and values for those columns based on JSON key-value pairs:
def extract_specific_values_as_columns(df, column, column_names_key='job', values_key='name', column_names=['Original Music Composer', 'Screenplay', 'Executive Producer']):
    # Function to convert JSON string to a dictionary of column_names_key: values_key pairs for specific column_names
    def convert_json(json_str):
        try:
            json_list = json.loads(json_str)
            values_dict = {item[column_names_key]: item[values_key] for item in json_list if item[column_names_key] in column_names}
            return values_dict
        except json.JSONDecodeError:
            return {}

    # Apply the function to the column
    df[column + '_json'] = df[column].apply(convert_json)

    # Create new columns for each column_name
    for column_name in column_names:
        df[column_name] = df[column + '_json'].apply(lambda x: x.get(column_name, ''))

    # Drop the original column and the intermediate '_json' column
    df = df.drop(columns=[column, column + '_json'])

    return df

In [ ]:
credits_columns_extracted = extract_specific_values_as_columns(credits_lists_extracted, 'crew', column_names_key='job', values_key='name', column_names=['Original Music Composer', 'Screenplay', 'Executive Producer'])


In [ ]:
print(credits_columns_extracted.head(20))
#print(credits_lists_extracted.head(20))
#print(credits.head(20))

    movie_id                                        title  \
0      19995                                       Avatar   
1        285     Pirates of the Caribbean: At World's End   
2     206647                                      Spectre   
3      49026                        The Dark Knight Rises   
4      49529                                  John Carter   
5        559                                 Spider-Man 3   
6      38757                                      Tangled   
7      99861                      Avengers: Age of Ultron   
8        767       Harry Potter and the Half-Blood Prince   
9     209112           Batman v Superman: Dawn of Justice   
10      1452                             Superman Returns   
11     10764                            Quantum of Solace   
12        58   Pirates of the Caribbean: Dead Man's Chest   
13     57201                              The Lone Ranger   
14     49521                                 Man of Steel   
15      2454     The Chr

In [ ]:
#Use the pandasql library to run sql queries
!pip install pandasql

  Preparing metadata (setup.py) ... done


 done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=4a2cc557c46bcb79eb4e993ef6adeb4c276795f711631aa19e02be7185cb886c
  Stored in directory: /home/noteable/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07


Successfully built pandasql


In [ ]:
#left join (i.e. augment) the final version of the movies dataset with the final version of the credits dataset:
from pandasql import sqldf

# Define a lambda function for convenience
pysqldf = lambda q: sqldf(q, globals())

query = """
SELECT m.*, c.* 
FROM movies_lists_extracted AS m 
LEFT JOIN credits_columns_extracted AS c
ON m.id=c.movie_id
"""

movies_merged = pysqldf(query)

In [ ]:
#inspect the merged dataset:
print(movies_merged.head(20))

       budget                                       genres  \
0   237000000  Action, Adventure, Fantasy, Science Fiction   
1   300000000                   Adventure, Fantasy, Action   
2   245000000                     Action, Adventure, Crime   
3   250000000               Action, Crime, Drama, Thriller   
4   260000000           Action, Adventure, Science Fiction   
5   258000000                   Fantasy, Action, Adventure   
6   260000000                            Animation, Family   
7   280000000           Action, Adventure, Science Fiction   
8   250000000                   Adventure, Fantasy, Family   
9   250000000                   Action, Adventure, Fantasy   
10  270000000  Adventure, Fantasy, Action, Science Fiction   
11  200000000           Adventure, Action, Thriller, Crime   
12  200000000                   Adventure, Fantasy, Action   
13  255000000                   Action, Adventure, Western   
14  225000000  Action, Adventure, Fantasy, Science Fiction   
15  2250

In [ ]:
#Export merged movies dataset to csv:
# Export the final merged DataFrame to a CSV file
movies_merged.to_csv('movies_merged.csv', index=False)